In [8]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-sds-j1lbPsoQT3bBiANjizMJH995OtfybS6Unih6v91_MA"

In [9]:
!pip install -q langchain-openai langchain-core requests

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [10]:
#tool create

@tool
def multiplication(a: int, b: int) -> int:
    """Multiply two integers"""
    return a * b


In [11]:
print(multiplication.invoke({"a": 2, "b": 3}))

6


In [12]:
multiplication.name

'multiplication'

In [13]:
multiplication.description

'Multiply two integers'

In [14]:
multiplication.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [15]:
llm = ChatOpenAI()

In [16]:
llm_with_tools = llm.bind_tools([multiplication])

In [17]:
llm_with_tools.invoke('hi how are you')

AIMessage(content="I'm here and ready to assist you! How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 49, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CGYJiHQjmnoQJ5s1YUM9tbd4eznpA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--78a6c877-4884-4751-a09e-b58b22b9a730-0', usage_metadata={'input_tokens': 49, 'output_tokens': 17, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
llm_with_tools.invoke('can you multiple 3 with 25')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hEcEVli1fefrLnPVUKD5Imvh', 'function': {'arguments': '{"a":3,"b":25}', 'name': 'multiplication'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 53, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CGYJjlldrc3btOLtiBX2RJ4LxdB6s', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0b9b6bac-2a72-4569-bb5a-a432de82a63a-0', tool_calls=[{'name': 'multiplication', 'args': {'a': 3, 'b': 25}, 'id': 'call_hEcEVli1fefrLnPVUKD5Imvh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 19, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache

In [19]:
query = HumanMessage('can you multiple 3 with 25')

In [20]:
messages = [query]

In [21]:
messages

[HumanMessage(content='can you multiple 3 with 25', additional_kwargs={}, response_metadata={})]

In [22]:
result = llm_with_tools.invoke(messages)

In [23]:
messages.append(result)

In [24]:
tool_result = multiplication.invoke(result.tool_calls[0])

In [25]:
messages.append(tool_result)

In [26]:
messages

[HumanMessage(content='can you multiple 3 with 25', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qLvXND83bpZOK4rVZRAasDyY', 'function': {'arguments': '{"a":3,"b":25}', 'name': 'multiplication'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 53, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CGYJkt8Qbj67FA6kBDNCmjBREW2ji', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f27b343d-0962-4bb3-b1d1-b983fcb2bd99-0', tool_calls=[{'name': 'multiplication', 'args': {'a': 3, 'b': 25}, 'id': 'call_qLvXND83bpZOK4rVZRAasDyY', 'type': 'tool_call'}], usage_metadata={'inpu

In [27]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 with 25 is 75.'

In [28]:
# tool create

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [29]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [30]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757980801,
 'time_last_update_utc': 'Tue, 16 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1758067201,
 'time_next_update_utc': 'Wed, 17 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.197}

In [31]:
convert.invoke({'base_currency_value':10, 'conversion_rate':88.197})

881.97

In [32]:
llm = ChatOpenAI()

In [33]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [34]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [35]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [36]:
ai_message = llm_with_tools.invoke(messages)

In [37]:
messages.append(ai_message)

In [38]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_V6RHPNsx6HzkH8GGEKmbAJgs',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_xmKFOAVLn0noDPA7uENie4P2',
  'type': 'tool_call'}]

In [39]:
import json
for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [40]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V6RHPNsx6HzkH8GGEKmbAJgs', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_xmKFOAVLn0noDPA7uENie4P2', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CGYJnErL9ZBdMes6awSdI6Wl8mk3q', 'service_tier': 'defa

In [41]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01134. \n\nBased on this conversion factor, the value of 10 INR is approximately 0.1134 USD.'

In [42]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [43]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here to assist you with any questions or tasks you have. How can I help you today?

> Finished chain.
